# Part 1

In [117]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Download the data!

In [122]:
postal_data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

In [123]:
postal_data.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [124]:
postal_data.shape

(180, 3)

### Drop the rows that contain "Not assigned" value in the 'Borough' column.

In [125]:
postal_data.drop(postal_data[ postal_data['Borough'] == "Not assigned" ].index, inplace = True)
postal_data.reset_index(drop=True, inplace=True)

In [126]:
postal_data.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Checking if theres any duplicate postal codes.

In [127]:
def check(df):
    for i in df.duplicated(subset=None, keep='first'):
        if i == True:
            return "There are duplicate postal codes!"
    return "There are no duplicate postal codes!"

check(postal_data)

'There are no duplicate postal codes!'

### Checking if a cell has a borough but a Not assigned neighborhood

In [128]:
postal_data[ postal_data['Neighborhood'] == "Not assigned"]

,Postal Code,Borough,Neighborhood


In [129]:
postal_data[ postal_data['Neighborhood'] == "NaN" ]

,Postal Code,Borough,Neighborhood


In [130]:
print("There are none!")

There are none!


In [131]:
postal_data.shape

(103, 3)

# Part 2

### Adding location data onto the data frame!

In [132]:
location_data = pd.read_csv('http://cocl.us/Geospatial_data')

In [133]:
location_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [137]:
location_data.shape

(103, 3)

In [138]:
postal_data = postal_data.merge(location_data)

In [139]:
postal_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [140]:
postal_data.shape

(103, 5)